In [2]:
import nltk
import spacy
stemmer = nltk.stem.porter.PorterStemmer()
spacy_nlp = spacy.load('en_core_web_sm')
doc = spacy_nlp(u'keyphrase generator')

import json
import os
from tqdm import tqdm

# hypers
dataset_names = ['kp20k', 'inspec', 'krapivin', 'semeval', 'duc', 'nus']
root_path = "/zf18/yw9fm/KPG_Project"
data_path = os.path.join(root_path,"data")

def summary_recall(name, test_data,groundtruth,beam_size):
    dec_dir = os.path.join(root_path, 'log', name, 'decode_model_495000_1587512303')
    gen_dir = os.path.join(dec_dir,'rouge_dec_dir')
    gen = os.listdir(gen_dir)
    gen = sorted(gen)
    result = []
    assert len(gen)==len(test_data)
    for idx, data in tqdm(enumerate(test_data)):
        if len(groundtruth[idx])>=1:
            ground = [' '.join([stemmer.stem(a) for a in ls]) for ls in groundtruth[idx]]
            read_path = os.path.join(gen_dir,gen[idx])
            assert os.path.exists(read_path)
            with open(read_path,'r') as f_out:
                tmp_sum = f_out.readlines()
            summ = [tmp.strip() for tmp in tmp_sum]
            assert len(summ)<=beam_size
            
            tmp=0
            text = ' '.join(summ)
            text = ' '.join([stemmer.stem(w) for w in text.split()])
            for gold in ground:
                if gold in text:
                    print(gold)
                    tmp += 1
            result.append(tmp*1.0/len(ground))
            
    print(sum(result)*1.0/len(result))
    return result

for name in ['kp20k']:
    test_path = os.path.join(data_path,name,name+'_test_spacynp.json')
    test_data = [json.loads(line) for line in open(test_path, 'r')]
    
#     present = [data['present_tgt_phrases'] for data in test_data]
#     print(name,'present')
#     summary_recall(name,test_data,present,100)
    print('absent')
    absent = [data['absent_tgt_phrases'] for data in test_data]
    result = summary_recall(name,test_data,absent,100)
    

keyphras
gener


In [11]:
from spacy.symbols import NOUN, PROPN, PRON
def get_noun_chunks(text, trim_punct=True, remove_stopword=True):
    spacy_doc = spacy_nlp(text, disable=["textcat"])
    np_chunks = list(spacy_doc.noun_chunks)
    np_str_list = []
    for chunk in np_chunks:
        np = []
        for w in chunk:
            w = w.text
            if trim_punct:
                w = w.strip(r"""!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~""")
            if remove_stopword:
                if w.lower() in stopword_set:
                    continue
            np.append(w)
        if len(np) > 0:
            np_str_list.append(' '.join(np))

    return np_str_list

def noun_chunks(doc, remove_duplicate=True):
    """
    Detect base noun phrases from a dependency parse. Works on both Doc and Span.
    """
    noun_chunk_list = []
    labels = ['nsubj', 'dobj', 'nsubjpass', 'pcomp', 'pobj', 'dative', 'appos',
              'attr', 'ROOT']
    id2name = {tid: t for tid, t in enumerate(spacy.symbols.NAMES)}
    np_deps = [doc.vocab.strings.add(label) for label in labels]
    conj = doc.vocab.strings.add('conj')
    np_set = set()

    for i, word in enumerate(doc):
        # print(i, word.text, id2name[word.pos], id2name[word.dep] if word.dep in id2name else 'np_dep')
        if word.pos not in (NOUN, PROPN, PRON):
            continue
        # Prevent nested chunks from being produced
        if word.dep in np_deps:
            # print(doc[word.left_edge.i: word.i+1])
            # print([id2name[t.pos] for t in doc[word.left_edge.i: word.i+1]])
            if remove_duplicate:
                for np in all_nested_NPs(doc[word.left_edge.i: word.i+1]):
                    if np.text not in np_set:
                        noun_chunk_list.append(np)
                        np_set.add(np.text)
            else:
                noun_chunk_list.extend(all_nested_NPs(doc[word.left_edge.i: word.i+1]))
        elif word.dep == conj:
            head = word.head
            while head.dep == conj and head.head.i < head.i:
                head = head.head
            # If the head is an NP, and we're coordinated to it, we're an NP
            if head.dep in np_deps:
                # print(doc[word.left_edge.i: word.i + 1])
                # print([id2name[t.pos] for t in doc[word.left_edge.i: word.i+1]])
                if remove_duplicate:
                    for np in all_nested_NPs(doc[word.left_edge.i: word.i+1]):
                        if np.text not in np_set:
                            noun_chunk_list.append(np)
                            np_set.add(np.text)
                else:
                    noun_chunk_list.extend(all_nested_NPs(doc[word.left_edge.i: word.i+1]))

    return noun_chunk_list
def all_nested_NPs(span):
    i = 0
    for i, word in enumerate(span):
        if word.pos != 89: # not a DET
            break

    span = span[i: ]
    nested_nps = []

    # a two-layer loop to get all possible nested phrases
    for k in range(1, len(span) + 1):
        for i in range(len(span) - k + 1):
            # print(span[i: i + k])
            np = span[i: i + k]
            nested_nps.append(np)

    return nested_nps
def stem_word_list(word_list):
    return [stemmer.stem(w.strip()) for w in word_list]

text = "A feedback vertex set of a graph G is a set S  of its vertices such that the subgraph induced by V(G)?S is a forest. The cardinality of a minimum feedback vertex set of G  is denoted by ?(G). A graph G is 2-degenerate  if each subgraph G? of G has a vertex v  such that dG?(v)?2. In this paper, we prove that ?(G)?2n/5 for any 2-degenerate n-vertex graph G and moreover, we show that this bound is tight. As a consequence, we derive a polynomial time algorithm, which for a given 2-degenerate n-vertex graph returns its feedback vertex set of cardinality at most 2n/5."
spacy_doc = spacy_nlp(text, disable=["textcat"])
spacy_nps = noun_chunks(spacy_doc, remove_duplicate=True)
nps = [[t.text.lower() for t in np] for np in spacy_nps]
stemmed_nps = [' '.join(stem_word_list(p)) for p in nps]
np_set = set(stemmed_nps)

# match_np = [p for p in np_set if p in present_tgts_set]
# recall = len(match_np) / len(stemmed_present_tgts) if len(stemmed_present_tgts) > 0 else -1.0

In [13]:
len(np_set)

101

In [20]:
def np_summary_recall(name, test_data,groundtruth,beam_size):
    dec_dir = os.path.join(root_path, 'log', name, 'decode_model_495000_1587512303')
    gen_dir = os.path.join(dec_dir,'rouge_dec_dir')
    gen = os.listdir(gen_dir)
    gen = sorted(gen)
    result = []
    assert len(gen)==len(test_data)
    for idx, data in tqdm(enumerate(test_data)):
        if len(groundtruth[idx])>=1:
            ground = [' '.join([stemmer.stem(a) for a in ls]) for ls in groundtruth[idx]]
            read_path = os.path.join(gen_dir,gen[idx])
            assert os.path.exists(read_path)
            with open(read_path,'r') as f_out:
                tmp_sum = f_out.readlines()
            summ = [tmp.strip() for tmp in tmp_sum[:beam_size]]
            assert len(summ)<=beam_size
            
            tmp=0
            text = ' '.join(summ)
            text = ' '.join([stemmer.stem(w) for w in text.split()])
            
            spacy_doc = spacy_nlp(text, disable=["textcat"])
            spacy_nps = noun_chunks(spacy_doc, remove_duplicate=True)
            nps = [[t.text.lower() for t in np] for np in spacy_nps]
            stemmed_nps = [' '.join(stem_word_list(p)) for p in nps]
            np_set = set(stemmed_nps)
            
            for gold in ground:
                if gold in np_set:
                    print(gold)
                    tmp += 1
            result.append(tmp*1.0/len(ground))
            
    print(sum(result)*1.0/len(result))
    return result

for name in ['kp20k']:
    test_path = os.path.join(data_path,name,name+'_test_spacynp.json')
    test_data = [json.loads(line) for line in open(test_path, 'r')]
    
#     present = [data['present_tgt_phrases'] for data in test_data]
#     print(name,'present')
#     summary_recall(name,test_data,present,100)
    print('absent')
    absent = [data['absent_tgt_phrases'] for data in test_data]
    result = np_summary_recall(name,test_data,absent,10)
    




0it [00:00, ?it/s]


1it [00:00,  7.89it/s]

absent





3it [00:00,  9.42it/s]


5it [00:00, 10.45it/s]


8it [00:00, 11.33it/s]


10it [00:00, 10.95it/s]


12it [00:00, 10.59it/s]


14it [00:01,  9.49it/s]


15it [00:01,  8.49it/s]


18it [00:01,  8.90it/s]


19it [00:02,  4.53it/s]


21it [00:02,  5.69it/s]


24it [00:02,  7.07it/s]


26it [00:02,  8.62it/s]


28it [00:02,  9.44it/s]


30it [00:02, 10.45it/s]


32it [00:12,  1.52s/it]

analysi of meet





33it [00:21,  3.82s/it]


34it [00:21,  2.71s/it]

...

hazard assess





297it [05:26,  2.52it/s]


299it [05:26,  3.11it/s]


300it [05:27,  3.25it/s]


304it [05:27,  3.90it/s]


305it [05:27,  4.02it/s]


306it [05:28,  4.87it/s]


310it [05:28,  6.44it/s]


312it [05:30,  2.37it/s]


313it [05:30,  3.03it/s]


314it [05:30,  3.53it/s]


316it [05:30,  4.52it/s]


318it [05:30,  5.46it/s]


322it [05:31,  6.41it/s]


324it [05:31,  6.46it/s]


326it [05:31,  8.09it/s]


328it [05:31,  8.10it/s]


330it [05:32,  7.83it/s]


331it [05:32,  8.27it/s]


333it [05:32,  9.06it/s]


335it [05:32,  9.73it/s]


337it [05:32,  9.84it/s]


339it [05:33,  8.67it/s]


340it [05:33,  3.51it/s]


343it [05:34,  4.62it/s]


345it [05:35,  2.49it/s]


348it [05:35,  3.35it/s]


351it [05:36,  4.38it/s]


353it [05:36,  5.15it/s]


356it [05:36,  6.71it/s]


358it [05:36,  8.34it/s]


360it [05:36,  9.99it/s]


362it [05:36,  9.49it/s]


364it [05:37,  6.99it/s]


366it [05:37,  7.13it/s]


368it [05:37,  7.57it/s]


369it [05:38,  2.44it/s]


370it [05:39,  3.01it/s]


perform analysi





555it [06:37,  1.76s/it]


556it [06:39,  2.07s/it]


557it [06:39,  1.49s/it]


559it [06:40,  1.11s/it]


560it [06:40,  1.22it/s]


561it [06:40,  1.59it/s]


562it [06:40,  2.08it/s]


563it [06:41,  2.68it/s]


564it [06:41,  2.51it/s]


565it [06:41,  3.17it/s]


566it [06:42,  2.40it/s]


567it [06:42,  2.56it/s]


568it [06:43,  1.78it/s]


569it [06:43,  2.04it/s]


570it [06:44,  2.57it/s]


571it [06:44,  3.23it/s]


573it [06:44,  4.18it/s]


574it [06:44,  4.76it/s]


575it [06:44,  5.51it/s]


578it [06:44,  7.10it/s]


581it [06:44,  8.56it/s]


584it [06:45,  8.38it/s]


586it [06:49,  1.30it/s]


587it [06:49,  1.72it/s]


588it [06:50,  1.93it/s]


589it [06:51,  1.31it/s]


590it [06:53,  1.07s/it]


593it [06:53,  1.31it/s]


594it [06:53,  1.75it/s]


595it [07:05,  3.95s/it]


596it [07:07,  3.28s/it]


597it [07:07,  2.33s/it]


598it [07:07,  1.66s/it]


600it [07:07,  1.21s/it]


601it [07:07,  1.14it/s]


602it [07:14,  2.48s/it]


603it [07:14,  1.77s/it]


KeyboardInterrupt: 

In [17]:
len(result)

16345